In [1]:
'''
Functions from in-class exercises
'''
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]
    
def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

In [2]:
'''
From project feedback. 
Credit: https://github.com/sunblaze-ucb/dpml-benchmark/blob/master/lossfunctions/logistic_regression.py#L12
'''

class LogisticRegression():
    @staticmethod
    def loss(theta, x, y, lambda_param=None):
        """Loss function for logistic regression with without regularization"""
        exponent = - y * (x.dot(theta))
        return np.sum(np.log(1+np.exp(exponent))) / x.shape[0]

    @staticmethod
    def gradient(theta, x, y, lambda_param=None):
        """
        Gradient function for logistic regression without regularization.
        Based on the above logistic_regression
        """
        exponent = y * (x.dot(theta))
        gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (
            x.shape[0])

        # Reshape to handle case where x is csr_matrix
        gradient_loss.reshape(theta.shape)

        return gradient_loss


class LogisticRegressionSinglePoint():
    @staticmethod
    def loss(theta, xi, yi, lambda_param=None):
        exponent = - yi * (xi.dot(theta))
        return np.log(1 + np.exp(exponent))

    @staticmethod
    def gradient(theta, xi, yi, lambda_param=None):

        # Based on page 22 of
        # http://www.cs.rpi.edu/~magdon/courses/LFD-Slides/SlidesLect09.pdf
        exponent = yi * (xi.dot(theta))
        return - (yi*xi) / (1+np.exp(exponent))


class LogisticRegressionRegular():
    @staticmethod
    def loss(theta, x, y, lambda_param):
        regularization = (lambda_param/2) * np.sum(theta*theta)
        return LogisticRegression.loss(theta, x, y) + regularization

    @staticmethod
    def gradient(theta, x, y, lambda_param):
        regularization = lambda_param * theta
        return LogisticRegression.gradient(theta, x, y) + regularization

In [3]:
'''
Functions taken from in-class-exercise 10.28.24
'''

# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

#Vectorized version of gradient calculation by wanglun1996. 
#Github: https://github.com/sunblaze-ucb/dpml-benchmark/blob/master/lossfunctions/logistic_regression.py#L12

def avg_grad(theta, X, y):

    #All_grads is a list of vectors, with each vector of length 104
    all_grads = [gradient(theta,X[i],y[i]) for i in range(len(X))] #one gradient per example in the data

    #Compute the column-wise average
    avg_grad = np.mean(all_grads,axis=0)
    
    return avg_grad

In [4]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [5]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


### IMPLEMENTING MINI-BATCH GRADIENT DESCENT (WITHOUT DP FIRST)

In [6]:
def split_to_mini_batches(X, y, batch_size):
    # shuffling the data before creating mini_batches to prevent the model 
    # from learning possible patterns + each batch might contain more "diversified"
    # information. 
    
    shuffled_data = np.random.permutation(X.shape[0])
    randomized_X = X[shuffled_data]
    randomized_Y = y[shuffled_data]

    mini_batches = []
    for i in range(0,X.shape[0],batch_size):
        mini_batches.append((randomized_X[i:i+batch_size], randomized_Y[i:i+batch_size]))
        
    return mini_batches

In [7]:
'''
Original function gradient_descent() taken from in-class-exercise 10.28.24 and modifying it
iterate over the mini-batches instead of over the entire dataset (full-batch)
'''
def mini_batch_gradient_descent(epochs, batch_size, learning_rate):
    #Step 1: initalize all thetas
    theta = [0 for _ in range(X_train.shape[1])] #Initial model

    #Step 2: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

    #Step 3: iterate for each num samples in training set (training_set = mini batch)
        for X_train_batch, y_train_batch in mini_batches:
            theta = theta - avg_grad(theta, X_train_batch, y_train_batch)
            
    return theta * learning_rate

#theta = mini_batch_gradient_descent(50 , 64, 0.1)
theta1 = mini_batch_gradient_descent(50 , 64, 0.01)
#theta
theta1

array([ 3.88150889e-05, -7.11876894e-03, -3.73156787e-03, -4.55750112e-03,
       -1.09982920e-02, -9.56543089e-03, -1.52240659e-02, -1.00876115e-02,
       -9.49865422e-03, -7.44231517e-03, -1.40846655e-02, -8.69202416e-03,
       -1.20928678e-02, -1.23791830e-02,  1.78450486e-04,  8.21645863e-04,
        6.14380856e-03,  1.13926793e-02, -2.34850491e-03,  8.97741210e-03,
       -1.47519222e-02,  1.36142668e-02, -9.07326343e-04, -1.50646836e-02,
        1.37998504e-02,  8.64071284e-03, -1.19817153e-02, -1.86414582e-02,
       -1.48703880e-02, -1.30391298e-02, -3.56952408e-03, -8.37051435e-04,
       -2.11940117e-03,  6.13951495e-03, -1.12320946e-02, -9.37175948e-03,
       -4.84457501e-03, -1.17159687e-02, -1.75592060e-02,  1.45503298e-03,
        2.70977577e-03,  2.88930435e-04,  3.30428779e-03, -3.80477297e-03,
       -9.98311924e-03, -5.50440499e-03, -1.38956850e-02, -1.60036173e-02,
       -6.51708784e-03,  7.47102686e-04, -1.26549276e-02, -6.45306132e-03,
       -1.16292985e-02, -

In [8]:
'''
Functions taken from in-class-exercise 10.28.24
'''
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias) #this is the dot product and take the sign. 
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2) #computing L2 norm 
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

theta = [-.1 for _ in range(104)]
accuracy(theta)

0.7585139318885449

### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH (EPSILON)- DP

In [9]:
def epsilon_noisy_gradient_descent(epochs, epsilon, batch_size,learning_rate=0.01):
    regression = LogisticRegression()
    
    #Step 1: initalize all thetas 
    theta = np.zeros(X_train.shape[1]) 


    #Step 2: splitting the epsilon and choosing sensitivity
    epsilon_i = epsilon/epochs
    sensitivity = 1 #?
    
    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [regression.gradient(theta,X_train_batch,y_train_batch) for i in range(len(X_train_batch))]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(all_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = laplace_mech_vec(grad_sum,sensitivity=sensitivity,epsilon=epsilon_i)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) 
            
            theta = theta - learning_rate * noisy_grad
    
    return theta

theta = epsilon_noisy_gradient_descent(10, 1.0, 64) #a smaller epsilon, accuracy is not as good. Noise can make the model worse. 
                                                    # If we increase iterations, it will make up for it. 


theta1 = epsilon_noisy_gradient_descent(10, 0.5, 64)
theta2 = epsilon_noisy_gradient_descent(10, 1.0, 55)
theta3 = epsilon_noisy_gradient_descent(10, 0.5, 55)
theta4 = epsilon_noisy_gradient_descent(10, 1.0, 70)
theta5 = epsilon_noisy_gradient_descent(10, 0.5, 70)

print('Final accuracy with epsilon = 1.0, epochs = 10, batch size = 64:', accuracy(theta))
print('Final accuracy with epsilon = 0.5, epochs = 10, batch size = 64: ', accuracy(theta1))
print('Final accuracy with epsilon = 1.0, epochs = 20, batch size = 55: ', accuracy(theta2))
print('Final accuracy with epsilon = 0.5, epochs = 20, batch size = 55: ', accuracy(theta3))
print('Final accuracy with epsilon = 1.0, epochs = 20, batch size = 70: ', accuracy(theta4))
print('Final accuracy with epsilon = 1.0, epochs = 20, batch size = 75: ', accuracy(theta5))


Final accuracy with epsilon = 1.0, epochs = 10, batch size = 64: 0.8149049093321539
Final accuracy with epsilon = 0.5, epochs = 10, batch size = 64:  0.8034055727554179
Final accuracy with epsilon = 1.0, epochs = 20, batch size = 55:  0.8184431667403803
Final accuracy with epsilon = 0.5, epochs = 20, batch size = 55:  0.8037372843874392
Final accuracy with epsilon = 1.0, epochs = 20, batch size = 70:  0.813578062804069
Final accuracy with epsilon = 1.0, epochs = 20, batch size = 75:  0.8166740380362671


### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH (EPSILON,DELTA)- DP

In [10]:
def vectorized_epsilon_delta_noisy_gradient_descent(epochs, epsilon, delta, batch_size,learning_rate=0.01):
    regression = LogisticRegression()
    
    #Step 1: initalize all thetas 
    theta = np.zeros(X_train.shape[1]) 

    #Step 2: splitting the epsilons and delta over the num of iterations/epochs.
    epsilon_i = epsilon/epochs
    delta_i = delta/epochs

    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [regression.gradient(theta,X_train_batch,y_train_batch) for i in range(len(X_train_batch))]
        
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_vec(grad_sum,sensitivity=b,epsilon=epsilon_i,delta=delta_i)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) 
            
            theta = theta - learning_rate * noisy_grad
    
    return theta


theta = vectorized_epsilon_delta_noisy_gradient_descent(10, 1.0, 1e-5,64)
theta1 = vectorized_epsilon_delta_noisy_gradient_descent(10, 0.5, 1e-5, 64)
theta2 = vectorized_epsilon_delta_noisy_gradient_descent(10, 1.0, 1e-5, 55)
theta3 = vectorized_epsilon_delta_noisy_gradient_descent(10, 0.5, 1e-5, 55)
theta4 = vectorized_epsilon_delta_noisy_gradient_descent(10, 1.0, 1e-5, 70)
theta5 = vectorized_epsilon_delta_noisy_gradient_descent(10, 0.5, 1e-5, 70)

print('Final accuracy with epsilon = 1.0, epochs = 10, batch size = 64:', accuracy(theta))
print('Final accuracy with epsilon = 0.5, epochs = 10, batch size = 64: ', accuracy(theta1))
print('Final accuracy with epsilon = 1.0, epochs = 20, batch size = 55: ', accuracy(theta2))
print('Final accuracy with epsilon = 0.5, epochs = 20, batch size = 55: ', accuracy(theta3))
print('Final accuracy with epsilon = 1.0, epochs = 20, batch size = 70: ', accuracy(theta4))
print('Final accuracy with epsilon = 1.0, epochs = 20, batch size = 75: ', accuracy(theta5))

Final accuracy with epsilon = 1.0, epochs = 10, batch size = 64: 0.7431446262715613
Final accuracy with epsilon = 0.5, epochs = 10, batch size = 64:  0.6372180451127819
Final accuracy with epsilon = 1.0, epochs = 20, batch size = 55:  0.7233524988942945
Final accuracy with epsilon = 0.5, epochs = 20, batch size = 55:  0.6501547987616099
Final accuracy with epsilon = 1.0, epochs = 20, batch size = 70:  0.6989164086687306
Final accuracy with epsilon = 1.0, epochs = 20, batch size = 75:  0.7390535161432994


### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH RÉNYI DP

In [19]:
'''
Original functions taken from homework assignment 9
'''
def vectorized_mini_batch_noisy_gradient_descent_RDP(epochs, epsilon_bar, alpha, batch_size,learning_rate=0.01):
    regression = LogisticRegression()
    
    #Step 1: initalize all thetas 
    theta = np.zeros(X_train.shape[1]) 

    epsilon_bar_i = epsilon_bar / epochs
    alpha_i = alpha/epochs


    #Step 3: split data into mini_batches
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [regression.gradient(theta,X_train_batch,y_train_batch) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_RDP_vec(grad_sum,sensitivity=b,alpha=alpha_i,epsilon=epsilon_bar_i)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) #MAYBE DO LEN(MINI_BATCH)
            
            theta = theta - learning_rate * noisy_grad
    
    return theta

theta = vectorized_mini_batch_noisy_gradient_descent_RDP(10, 0.1, 20, 64)
theta1 = vectorized_mini_batch_noisy_gradient_descent_RDP(10, 0.3, 20, 64)
theta2 = vectorized_mini_batch_noisy_gradient_descent_RDP(10, 0.3, 20, 55)
theta3 = vectorized_mini_batch_noisy_gradient_descent_RDP(10, 0.1, 15, 55)
theta4 = vectorized_mini_batch_noisy_gradient_descent_RDP(10, 0.1, 15, 70)
theta5 = vectorized_mini_batch_noisy_gradient_descent_RDP(10, 0.1, 25, 70)

print('Final accuracy with epsilon_bar = 0.1, epochs = 10, alpha = 20, batch size = 64:', accuracy(theta))
print('Final accuracy with epsilon_bar = 0.2, epochs = 10, alpha = 20, batch size = 64: ', accuracy(theta1))
print('Final accuracy with epsilon_bar = 0.3, epochs = 10, alpha = 20, batch size = 55: ', accuracy(theta2))
print('Final accuracy with epsilon_bar = 0.3, epochs = 10, alpha = 15, batch size = 55: ', accuracy(theta3))
print('Final accuracy with epsilon_bar = 0.1, epochs = 10, alpha = 15, batch size = 70: ', accuracy(theta4))
print('Final accuracy with epsilon_bar = 0.1, epochs = 10, alpha = 25, batch size = 70: ', accuracy(theta5))

Final accuracy with epsilon_bar = 0.1, epochs = 10, alpha = 20, batch size = 64: 0.8069438301636445
Final accuracy with epsilon_bar = 0.2, epochs = 10, alpha = 20, batch size = 64:  0.8224237063246351
Final accuracy with epsilon_bar = 0.3, epochs = 10, alpha = 20, batch size = 55:  0.8186643078283945
Final accuracy with epsilon_bar = 0.3, epochs = 10, alpha = 15, batch size = 55:  0.8094869526758072
Final accuracy with epsilon_bar = 0.1, epochs = 10, alpha = 15, batch size = 70:  0.8146837682441398
Final accuracy with epsilon_bar = 0.1, epochs = 10, alpha = 25, batch size = 70:  0.8099292348518354


### IMPLEMENTING MINI-BATCH GRADIENT DESCENT WITH zCDP

In [18]:
'''
Original functions taken from homework assignment 9
'''
def vectorized_mini_batch_noisy_gradient_descent_zCDP(epochs, rho, batch_size,learning_rate=0.01):
    #IDEA: copy noisy_gradient_descent but use gaussian_mech_zCDP_vec to compute the noisy_gradient_sum
    regression = LogisticRegression()
    rho_i = rho/ epochs
    
    #from the noisy_gradient_descent function provided above: 
    theta = np.zeros(X_train.shape[1])
    for _ in range(epochs): #epochs = iterations
        mini_batches = split_to_mini_batches(X_train, y_train, batch_size)

        for X_train_batch, y_train_batch in mini_batches:
        
            all_grads = [regression.gradient(theta,X_train_batch,y_train_batch) for i in range(len(X_train_batch))]
            
            # 2. Call L2_clip on each gradient
            b = 3
            clipped_grads = [L2_clip(g, b) for g in all_grads]
            
            # 3. Take the sum of the clipped gradients and add noise
            grad_sum = np.sum(clipped_grads, axis=0)
    
            #Sensitivity is correct, by clipping
            noisy_grad_sum = gaussian_mech_zCDP_vec(grad_sum,sensitivity=b,rho=rho_i)
    
            noisy_grad = np.array(noisy_grad_sum )/ len(X_train_batch) #Danger: reveals the size of the training data (probably not a big deal but
            # does violate DP) #MAYBE DO LEN(MINI_BATCH)?
            
            theta = theta - learning_rate * noisy_grad
    
    return theta

theta = vectorized_mini_batch_noisy_gradient_descent_zCDP(10, 0.1, 64)
theta1 = vectorized_mini_batch_noisy_gradient_descent_zCDP(10, 0.2, 64)
theta2 = vectorized_mini_batch_noisy_gradient_descent_zCDP(10, 0.1, 55)
theta3 = vectorized_mini_batch_noisy_gradient_descent_zCDP(10, 0.2, 55)
theta4 = vectorized_mini_batch_noisy_gradient_descent_zCDP(10, 0.1, 70)
theta5 = vectorized_mini_batch_noisy_gradient_descent_zCDP(10, 0.2, 70)

print('Final accuracy with rho = 1.0, epochs = 10, batch size = 64', accuracy(theta))

print('Final accuracy with rho = 0.5, epochs = 10, batch size = 64: ', accuracy(theta1))
print('Final accuracy with rho = 1.0, epochs = 20, batch size = 55: ', accuracy(theta2))
print('Final accuracy with rho = 0.5, epochs = 20, batch size = 55: ', accuracy(theta3))
print('Final accuracy with rho = 1.0, epochs = 20, batch size = 70: ', accuracy(theta4))
print('Final accuracy with rho = 1.0, epochs = 20, batch size = 75: ', accuracy(theta5))

Final accuracy with rho = 1.0, epochs = 10, batch size = 64 0.8120300751879699
Final accuracy with rho = 0.5, epochs = 10, batch size = 64:  0.8171163202122954
Final accuracy with rho = 1.0, epochs = 20, batch size = 55:  0.8151260504201681
Final accuracy with rho = 0.5, epochs = 20, batch size = 55:  0.8244139761167625
Final accuracy with rho = 1.0, epochs = 20, batch size = 70:  0.8162317558602389
Final accuracy with rho = 1.0, epochs = 20, batch size = 75:  0.8154577620521893
